In [1]:
import sys, os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time 

import networkx as nx


In [2]:
dataset = 'sp100' 

os.chdir("../data_modified")

corr_tensor = np.load('%s_corr.npy' % (dataset))
dates = np.load('%s_dates.npy' % (dataset))
nodes = np.load('%s_nodes.npy' % (dataset))

num_examples = corr_tensor.shape[0] #number of dates
dim = corr_tensor.shape[1] #number of assets


In [3]:
# Helper Functions

def make_graph(corr_mat, node_labels, graph_type):

    G = nx.Graph()
    G.add_nodes_from(node_labels)
    dim = corr_mat.shape[0]

    if not dim == len(node_labels):
        raise ValueError('number node labels not = corr matrix dimensions')

    if graph_type=='signed':
        for i in range(dim):
            for j in range(i+1, dim):
                if corr_mat[i,j] < 0:
                    G.add_edge(node_labels[i], node_labels[j], sign=-1)
                elif corr_mat[i,j] > 0:
                    G.add_edge(node_labels[i], node_labels[j], sign=1)
    
    if graph_type=='corr':
        for i in range(dim):
            for j in range(i+1, dim):
                if corr_mat[i,j] != 0.000:
                    G.add_edge(node_labels[i], node_labels[j])
    
    if graph_type=='uncorr':
        for i in range(dim):
            for j in range(i+1, dim):
                if corr_mat[i,j] == 0.000:
                    G.add_edge(node_labels[i], node_labels[j])
    
    density = (2*G.number_of_edges())/(G.number_of_nodes()*(G.number_of_nodes() - 1))
                
    return G, density

def get_max_deg(G):
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)

    return max(degree_sequence)

def coloring_score(G, coloring):
    count = 0
    for e in G.edges():
        v1, v2 = e
        if coloring[v1] == coloring[v2]:
            count += 1
        
        return count/G.number_of_edges()

In [4]:
print("num examples: %d, matrix dim: %d" % (num_examples, dim))

num examples: 42, matrix dim: 90


In [5]:
corr_mat = corr_tensor[int(num_examples/2), :, :].copy()

corr_mat[(corr_mat > -1*0.5) & (corr_mat < 0.5)] = 0
G, density = make_graph(corr_mat, nodes, 'corr')

In [8]:
# Run classical graph coloring algm 
from networkx.algorithms.coloring import greedy_color

coloring_array = []
num_colors_array = []
date_array = []
density_array = []
time_array = []
threshold_array = []


count = 0
for i in np.arange(0.5, 1, 0.1):
    for j in range(1, int(num_examples/5)):
        
        corr_mat = corr_tensor[j*5, :, :].copy()
        corr_mat[(corr_mat > -1*i) & (corr_mat < i)] = 0
        
        G, density = make_graph(corr_mat, nodes, 'corr')
        
        count += 1
        if count % 10 == 0: 
            print("count: %d" % (count))
    try:
        t = time.clock()
        coloring = greedy_color(G, strategy='independent_set')
        computation_time = time.clock() - t
    except Exception as err:
        print("Error on matrix %d with threshold %f" % (j*5, i))
    else:
        num_colors = np.max(list(coloring.values())) + 1

        coloring_array.append(coloring)
        num_colors_array.append(num_colors)
        time_array.append(computation_time)
        density_array.append(density)
        threshold_array.append(i)
        date_array.append(dates[j*5])
            

count: 10
count: 20
count: 30


In [10]:
os.chdir("../result_files")

# Create Pandas DataFrame for classical results
pd.DataFrame(data={"date": date_array, "threshold": threshold_array, "density": density_array,
                "coloring": coloring_array, "num_colors": num_colors_array, 
                }).to_csv("grphcolor_class_%s_res.csv" % (dataset))